In [ ]:
import collections
import argparse
from tqdm import tqdm
import json
import torch

from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased') #/wiki-bert-lt-cased
model = BertModel.from_pretrained("bert-base-multilingual-cased") #/wiki-bert-lt-cased
text = read_examples("ObjCase.txt")
with open("ObjCase_multi.json", "w", encoding='utf-8') as writer: #/ObjCase_lt.json
    for sentence in text:
        encoded_input = tokenizer(sentence, return_tensors='pt', padding = True)
        output = model(**encoded_input, output_hidden_states = True)
        hidden_states = output[2][1:]
        output_json = collections.OrderedDict()
        output_json["linex_index"] = text.index(sentence)
        all_out_features = []
        all_layers = []
        for layer_index in range(len(hidden_states)):
            layer_output = hidden_states[int(layer_index)].detach().cpu().numpy()
            layer_output = layer_output[0, 0, :]
            layers = collections.OrderedDict()
            layers["index"] = layer_index
            layers["values"] = [
                round(x.item(), 6) for x in layer_output
            ]
            all_layers.append(layers)
        out_features = collections.OrderedDict()
        out_features["token"] = "[CLS]"
        out_features["layers"] = all_layers
        all_out_features.append(out_features)
        output_json["features"] = all_out_features
        writer.write(json.dumps(output_json) + "\n")

In [ ]:
def read_examples(input_file):
  """Read a list of `InputExample`s from an input file."""
  examples = []
  unique_id = 0
  with open(input_file, "r", encoding='utf-8') as reader:
    while True:
      line = reader.readline()
      if not line:
          break
      text = line.strip().split('\t')[-1]
      examples.append(text)
      unique_id += 1
  return examples


KeyboardInterrupt

